In [1]:
using Pkg
Pkg.add(;path="/Users/az/Documents/GitHub/GLOM_RV_Example")

   Updating git-repo `/Users/az/Documents/GitHub/GLOM_RV_Example`
   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
No Changes to `~/Documents/GitHub/GLOM_RV_Example/examples/Project.toml`
No Changes to `~/Documents/GitHub/GLOM_RV_Example/examples/Manifest.toml`


In [2]:

Pkg.activate("examples")
Pkg.instantiate()

 Activating environment at `~/Documents/GitHub/GLOM_RV_Example/examples/examples/Project.toml`


In [3]:
import Pkg; Pkg.add("CSV")

using Statistics

# for det()
using LinearAlgebra

# for importing the data from CSV
using DataFrames
using CSV


  Resolving package versions...
No Changes to `~/Documents/GitHub/GLOM_RV_Example/examples/examples/Project.toml`
No Changes to `~/Documents/GitHub/GLOM_RV_Example/examples/examples/Manifest.toml`


In [4]:
# For GLOM
# import Pkg; Pkg.add("GPLinearODEMaker")
import GPLinearODEMaker; GLOM = GPLinearODEMaker

GPLinearODEMaker

In [5]:
# For this module
using GLOM_RV_Example; GLOM_RV = GLOM_RV_Example

GLOM_RV_Example

In [6]:
# For units in orbit fitting functions
import Pkg; Pkg.add("Unitful")
using UnitfulAstro, Unitful

  Resolving package versions...
No Changes to `~/Documents/GitHub/GLOM_RV_Example/examples/examples/Project.toml`
No Changes to `~/Documents/GitHub/GLOM_RV_Example/examples/examples/Manifest.toml`


In [7]:

## Problem setup

# CHANGE: choose a kernel, I suggest 3 for Matern 5/2 or 4 for Quasi-periodic
# kernel
kernel_choice = 4
kernel_names = ["pp", "se", "m52", "qp", "m52_m52", "se_se"]
kernel_name = kernel_names[kernel_choice]
kernel_function, num_kernel_hyperparameters = GLOM.include_kernel(kernel_name)

(GPLinearODEMaker.qp_kernel, 3)

In [8]:
# CHANGE: the stars rotation rate which is used as the first guess for some GLOM
# hyperparameters and starting point for priors
star_rot_rate = 27.  # days for solar rotation

# CHANGE: observation times go here
using DelimitedFiles
obs_xs = collect(Iterators.flatten(readdlm("./data/bjd_daily.txt")))
# taking out the mean observation times makes the optimization easier for models
# with periodic parameters
GLOM_RV.remove_mean!(obs_xs)

567-element Array{Float64,1}:
 -521.6919832368294
 -520.693027812893
 -519.6935705179712
 -518.6966490866398
 -517.6936680239596
 -516.6960158565344
 -515.6896057711201
 -514.7484853224014
 -512.6859100910442
 -502.68925038841553
 -501.70853298048314
 -500.7112448332482
 -499.663915724188
    ⋮
  546.2984948420926
  547.3109326377598
  548.2987155717055
  549.298295535642
  551.3166310471352
  552.2934769574713
  553.2954330905413
  556.3016304768971
  558.3333976976355
  559.3063403805936
  560.3000211960316
  561.2330296329455

In [9]:
# CHANGE: rvs and their errors go here
obs_rvs = collect(Iterators.flatten(readdlm("./data/rv_daily.txt")))
# inject_ks = GLOM_RV.kep_signal(; K=50u"m/s", P=sqrt(2)*5u"d", M0=rand()*2*π, ω_or_k=rand()*2*π, e_or_h=0.1)
# obs_rvs[:] .+= ustrip.(inject_ks.(obs_xs.*u"d"))
obs_rvs_err = collect(Iterators.flatten(readdlm("./data/erv_daily.txt")))

567-element Array{Float64,1}:
 0.03036812271298431
 0.028174782382301478
 0.02938168070678178
 0.030733648918260983
 0.030273630749804166
 0.02978809045216476
 0.03479668670704809
 0.05661399795951553
 0.049404085779396215
 0.02941258553740282
 0.07400877379839974
 0.05641940799521519
 0.05493576014838747
 ⋮
 0.035898019589951376
 0.03852508007629684
 0.034726130635685894
 0.06653000613481366
 0.03769443757625897
 0.03762794120703391
 0.08524877149613588
 0.08505459459542762
 0.05667514069294367
 0.040044931610342276
 0.040362126578561225
 0.07987414815014192

In [10]:
# CHANGE: activity indicators and thier errors go here
# you can actually have as many as you want, but obviously it will take longer
# to fit
obs_indicator1 = collect(Iterators.flatten(readdlm("./data/C1.txt")))
obs_indicator1_err = collect(Iterators.flatten(readdlm("./data/err_C1.txt")))
obs_indicator2 = collect(Iterators.flatten(readdlm("./data/C2.txt")))
obs_indicator2_err = collect(Iterators.flatten(readdlm("./data/err_C2.txt")))


567-element Array{Float64,1}:
  5.756971312004282
  5.3652157914954595
  5.654645807928711
  5.925730235674198
  5.825554595635701
  5.887501707941508
  6.627535360942655
 10.619782436423465
  9.48548099736605
  5.536845159896076
 13.636395158606499
 10.394849664742399
 10.086845919158254
  ⋮
  6.43037650634545
  6.968414949269418
  6.354296665094809
 11.94942703898325
  6.845679613674609
  6.914610698625592
 15.278262565559656
 15.088280249291216
 10.530894994919043
  7.371796212363171
  7.278971515511814
 14.141866792926344

In [11]:
# removing means as the GP model assumes zero mean
GLOM_RV.remove_mean!(obs_rvs)
GLOM_RV.remove_mean!(obs_indicator1)
GLOM_RV.remove_mean!(obs_indicator2)

567-element Array{Float64,1}:
  -9.19469615309195
  -2.181219040108357
  24.36547134951356
  11.572529475741238
  15.839708855084286
   6.184546259448797
  -2.3319649315486233
  -6.962528319440719
   7.891000324704993
 -12.15126452117482
  -6.724323015519727
  -3.605547890793848
  -1.1283323401620011
   ⋮
  30.61435175253286
  20.18970107755273
  32.69960257219914
  22.37319327818817
  10.755748439616642
  19.025340456475934
  22.446504635230657
  55.76007082977286
  33.29974799399908
  17.139171367331546
  21.66094258543787
   9.328425393001641

In [12]:
# CHANGE: change these lines if you add more than 2 indicators
# this takes the data and riffles it together so it takes the form
# [rv_1, ind1_1, ind2_1, rv_2, ind1_2, ind2_2, ...]
n_out = 3  # number of indicators + 1
obs_ys = collect(Iterators.flatten(zip(obs_rvs, obs_indicator1, obs_indicator2)))
obs_noise = collect(Iterators.flatten(zip(obs_rvs_err, obs_indicator1_err, obs_indicator2_err)))


1701-element Array{Float64,1}:
  0.03036812271298431
  7.86437518965105
  5.756971312004282
  0.028174782382301478
  8.077654743316922
  5.3652157914954595
  0.02938168070678178
  7.700195206719579
  5.654645807928711
  0.030733648918260983
  7.8474301601074075
  5.925730235674198
  0.030273630749804166
  ⋮
  0.05667514069294367
 16.321341796880354
 10.530894994919043
  0.040044931610342276
 11.004962966143173
  7.371796212363171
  0.040362126578561225
 11.017223811322138
  7.278971515511814
  0.07987414815014192
 21.68829497563989
 14.141866792926344

In [13]:
# How many differention orders we want in the GLOM model
n_dif = 2 + 1

3

In [14]:
# CHANGE: consider changing a0 (the GLOM coefficients that are used, see
# commented lines below)
# If all a's active:
problem_definition = GLOM.GLO(kernel_function, num_kernel_hyperparameters, n_dif, n_out, obs_xs, copy(obs_ys); noise=copy(obs_noise), a0=[[1. 1 1];[1 1 1];[1 1 1]])
# problem_definition = GLOM.GLO(kernel_function, num_kernel_hyperparameters, n_dif, n_out, obs_xs, copy(obs_ys); noise=copy(obs_noise), a0=[[1. 1 0];[1 0 1];[1 0 1]])

# Makes the std of each output equal to 1, improves fitting stability
# the normalizations are stored in problem_definition.normals
GLOM.normalize_problem_definition!(problem_definition)

In [15]:
# CHANGE: Setting initial fit values
initial_total_hyperparameters = collect(Iterators.flatten(problem_definition.a0))
initial_hypers = [[star_rot_rate], [star_rot_rate], [star_rot_rate], [star_rot_rate, 2 * star_rot_rate, 1], [star_rot_rate, 2 * star_rot_rate, 1], [star_rot_rate, 2 * star_rot_rate, 1]]
append!(initial_total_hyperparameters, initial_hypers[kernel_choice])

12-element Array{Float64,1}:
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
 27.0
 54.0
  1.0

In [16]:
## Fitting GLOM Model

# CHANGE: Setting kernel hyperparameter priors and kick function
# kick functions help avoid saddle points
tighten_lengthscale_priors = 3
if kernel_name in ["pp", "se", "m52"]
    kernel_hyper_priors(hps::Vector{<:Real}, d::Integer) =
        GLOM_RV.kernel_hyper_priors_1λ(hps, d, star_rot_rate, star_rot_rate / 2 / tighten_lengthscale_priors)
    add_kick!(hps::Vector{<:Real}) = GLOM_RV.add_kick_1λ!(hps)
elseif kernel_name == "qp"
    kernel_hyper_priors(hps::Vector{<:Real}, d::Integer) =
        GLOM_RV.kernel_hyper_priors_qp(hps, d, [star_rot_rate, 2 * star_rot_rate, 1], [star_rot_rate / 2, star_rot_rate / 2, 0.4] ./ tighten_lengthscale_priors)
    add_kick!(hps::Vector{<:Real}) = GLOM_RV.add_kick_qp!(hps)
elseif kernel_name in ["se_se", "m52_m52"]
    kernel_hyper_priors(hps::Vector{<:Real}, d::Integer) =
        GLOM_RV.kernel_hyper_priors_2λ(hps, d, [star_rot_rate, 2 * star_rot_rate, 1], [star_rot_rate / 2, star_rot_rate / 2, 1] ./ tighten_lengthscale_priors)
    add_kick!(hps::Vector{<:Real}) = GLOM_RV.add_kick_2λ!(hps)
else
    # kernel_hyper_priors(hps::Vector{<:Real}, d::Integer) = custom function
end




add_kick! (generic function with 1 method)

In [ ]:
fit1_total_hyperparameters, result = GLOM_RV.fit_GLOM(problem_definition, initial_total_hyperparameters, kernel_hyper_priors, add_kick!)
# fit_GLOM returns a vector of num_kernel_hyperparameters gp hyperparameters
# followed by the GLOM coefficients and the Optim result object
workspace = GLOM.nlogL_matrix_workspace(problem_definition, fit1_total_hyperparameters)

gridsearching over hp10
starting at hp = 27.0 -> 55933.88821624688
searching from 20.25 to 33.75
ending at hp   = 28.35 -> 53905.66879537845
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 28.35, 54.0, 1.0]

[1.074738454118229, 0.7102617549534178, 0.7941053271075498, 1.1428292246520722, 0.9931206101157951, 0.9817883149305041, 1.5039917546181174, 1.0044360018110856, 1.0035007550524033, 28.380433309984245, 53.954433547720576, 1.7682786350874862]

Iteration:              1
Time so far:            2165.762631893158 s
Unnormalized posterior: 25831.643146116367
Gradient norm:          24296.46769581849

[1.1619719162997066, 0.5013605417272822, 0.5222792628581263, 1.268171550590119, 1.043587978066286, 1.0348321003063807, 1.712871556305054, 0.9404148974356956, 1.0164292063650464, 30.432816098781014, 53.93804793846952, 2.3579022227075566]

Iteration:              2
Time so far:            4374.924706935883 s
Unnormalized posterior: 15161.63691299514
Gradient norm:          11835.678727886629

[0.

In [ ]:
## Plotting initial results

plot_xs = collect(LinRange(obs_xs[1]-10, obs_xs[end]+10, 300))
post, post_err, post_obs = GLOM_RV.GLOM_posteriors(problem_definition, plot_xs, fit1_total_hyperparameters)
GLOM_rvs_at_plot_xs, GLOM_ind1_at_plot_xs, GLOM_ind2_at_plot_xs = post
GLOM_rvs_err_at_plot_xs, GLOM_ind1_err_at_plot_xs, GLOM_ind2_err_at_plot_xs = post_err
GLOM_rvs_at_obs_xs, GLOM_ind1_at_obs_xs, GLOM_ind2_at_obs_xs = post_obs